# 1 Data

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from IPython.display import display
import gc
import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

%matplotlib inline
import matplotlib.pyplot as plt

import torch
from torch import optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

import kornia as K
from kornia import image_to_tensor, tensor_to_image
import kornia.augmentation as aug

from transformers import get_cosine_schedule_with_warmup

In [ ]:
seed = 402
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

# 1 Data

norm_score = [0.01, 1.0]

the output will be in (0.005, 1.005)

sigmoid(y_hat) + 0.005

In [ ]:
class PetData(Dataset):
    def __init__(self, df, transform, is_test=False):
        super(PetData, self).__init__()
        self.binary_features = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
        self.df = df
        self.transform = transform
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row["path"]
        img = Image.open(image_path).convert("RGB")
        data = self.transform(img)
        binary_features = torch.tensor(row[self.binary_features], dtype=torch.long)
        if self.is_test:
            return data, binary_features
        else:
            label = torch.tensor(row["norm_score"], dtype=torch.float) # [0.01, 1.0]
            return data, label, binary_features

In [ ]:
class Data:
    def __init__(self, batch_size=16, img_size=224, n_split=10):
        self.batch_size = batch_size
        self.img_size = img_size
        
        self.train_val_df = pd.read_csv("../input/petfinder-data-with-10-folds/train_val_df.csv")
        self.test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
        
        image_mean = [0.38753143, 0.36847523, 0.27735737]
        image_std = [0.25998375, 0.23844026, 0.2313706]
        normTransform = T.Normalize(image_mean, image_std)
        self.trainTransform = T.Compose([
#             T.RandomResizedCrop((img_size, img_size), (0.8, 1.0)),
            T.Resize((img_size+32, img_size+32)),
            T.CenterCrop((img_size, img_size)),
#             T.Resize((img_size, img_size)),
            T.RandomRotation(30),
            T.RandomHorizontalFlip(),
            T.ColorJitter(0.2, 0.2, 0.2, 0.1),
            T.ToTensor(),
            normTransform,
            T.RandomErasing(scale=(0.02, 0.13)),
        ])
        self.validTransform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            normTransform
        ])        
        self.num_workers = 2
        
    def train_dataloader(self, fold):
        train_df = self.train_val_df.query(f'fold != {fold}')
        train_ds = PetData(train_df, self.trainTransform, is_test=False)
        train_dl = DataLoader(train_ds, batch_size=self.batch_size, shuffle=True, pin_memory=True, num_workers=self.num_workers)
        return train_dl
        
    def val_dataloader(self, fold):
        val_df = self.train_val_df.query(f'fold == {fold}')
        val_ds = PetData(val_df, self.validTransform, is_test=False)
        val_dl = DataLoader(val_ds, batch_size=self.batch_size*2, shuffle=False, pin_memory=True, num_workers=self.num_workers)
        return val_dl
        
    def test_dataloader(self):
        test_ds = PetData(self.test_df, self.validTransform, is_test=True)
        test_dl = DataLoader(test_ds, batch_size=self.batch_size*2, shuffle=False, pin_memory=True, num_workers=self.num_workers)
        return test_dl

# 2 Model

In [ ]:
class Model(nn.Module):
    def __init__(self, backbone_name, binary_features=['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']):
        super(Model, self).__init__()
        self.backbone = create_model(model_name, pretrained=True, num_classes=64)
        self.feat_embeddings = nn.ModuleList(nn.Embedding(2, 1) for _ in range(len(binary_features)))
        self.fc = nn.Linear(64 + 1*len(binary_features), 1, bias=True)
    
    def forward(self, x, feats):
        x = self.backbone(x)
#         return torch.sigmoid(x) + 0.005 # (0.005, 1.005)
        feats_embedding_list = []
        for idx in range(len(self.feat_embeddings)):
            embedding_model = self.feat_embeddings[idx]
            embedding_input = feats[:, idx]
            feats_embedding_list.append(embedding_model(embedding_input))
        feats_embedding = torch.cat(feats_embedding_list, dim=-1)
        return torch.sigmoid(self.fc(torch.cat([x, feats_embedding], dim=-1))) + 0.005 # the output will be (0.005, 1.005) has the scope of [0.01, 1.].
        

# 3 Train

In [ ]:
from torch.cuda.amp import autocast as autocast

In [ ]:
class Trainer:
    def __init__(self, num_epochs, lr, wd):
        self.loss_func = nn.MSELoss()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_epochs = num_epochs
        self.lr = lr
        self.wd = wd
        self.min_loss = 1e3
        
    def get_fold_data(self, data, fold):
        train_dl = data.train_dataloader(fold=fold)
        val_dl = data.val_dataloader(fold=fold)
        return train_dl, val_dl
    
    def train_fold(self, data, fold, model_name='swin_large_patch4_window7_224'):
        train_dl, val_dl = self.get_fold_data(data, fold)
        model = Model(model_name)
        model.to(self.device)
        no_decay = ["bias", "LayerNorm.weight", "LayerNorm.bias"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.wd,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(optimizer_grouped_parameters, lr=self.lr)
        num_training_steps = self.num_epochs * len(train_dl)
        num_warmup_steps = int(0.1 * num_training_steps)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
        train_loss_list = [None] * num_training_steps
        for epoch in range(self.num_epochs):
            tbar = tqdm(train_dl)
            epoch_loss = 0.
            train_step = 0
            model.train()
            for batch_idx, batch in enumerate(tbar):
                x, y, feats = batch
                x, y, feats = x.to(self.device), y.to(self.device), feats.to(self.device)
                with autocast():    
                    y_hat = model(x, feats).squeeze()
                    loss = self.loss_func(y_hat, y)
                    loss = torch.sqrt(loss)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
                
                train_loss_list[train_step] = loss.detach().cpu().numpy() # petfinder_rmse(y_hat, y)
                epoch_loss += train_loss_list[train_step]
                train_step += 1
                
                tbar.set_description_str("[Train Epoch %d/%d]"% (epoch+1, self.num_epochs))
                tbar.set_postfix_str("Loss: %.5f, lr: %.6f"% (100 * epoch_loss / train_step, scheduler.get_last_lr()[0]))
                
                del loss, x, y, y_hat, feats
                gc.collect()
                
            val_loss = self.eval_step(val_dl, model, epoch)
            if val_loss <= self.min_loss:
                self.min_loss = val_loss
                torch.save(model.state_dict(), "model_fold%d.pth"%fold)
            
        del train_dl, val_dl, model, optimizer, scheduler
        gc.collect()
        return train_loss_list
    
    def eval_step(self, val_dl, model, epoch):
        tbar = tqdm(val_dl)
        epoch_loss = 0.
        eval_step = 0
        model.eval()
        for batch_idx, batch in enumerate(tbar):
            x, y, feats = batch
            x, y, feats = x.to(self.device), y.to(self.device), feats.to(self.device)
            with torch.no_grad():
                y_hat = model(x, feats).squeeze()
            loss = self.loss_func(y_hat, y)
            epoch_loss += torch.sqrt(loss).detach().cpu().numpy()
            eval_step += 1
            
            tbar.set_description_str("[Val Epoch %d]"% (epoch+1))
            tbar.set_postfix_str("Loss: %.5f"% (100 * epoch_loss / eval_step))
            
            del x, y, y_hat
            gc.collect()
        
        return epoch_loss / eval_step

```
model_names = [
    'swin_large_patch4_window7_224',
    'swin_large_patch4_window7_224_in22k',
    'swin_large_patch4_window12_384',
    'swin_large_patch4_window12_384_in22k'
]
```

In [ ]:
num_epochs, lr, wd = 5, 2e-5, 5e-4
n_split = 10
img_size = 224 # 384
with_22k = False

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
if img_size == 224:
    if with_22k:
        model_name = 'swin_large_patch4_window7_224_in22k'
        !cp ../input/swin-large-models/swin_large_patch4_window7_224_22k.pth /root/.cache/torch/hub/checkpoints/
    else:
        model_name = 'swin_large_patch4_window7_224'
        !cp ../input/swin-large-models/swin_large_patch4_window7_224_22kto1k.pth /root/.cache/torch/hub/checkpoints/
    batch_size = 32
else:
    if with_22k:
        model_name = 'swin_large_patch4_window12_384_in22k'
        !cp ../input/swin-large-models/swin_large_patch4_window12_384_22k.pth /root/.cache/torch/hub/checkpoints/
    else:
        model_name = 'swin_large_patch4_window12_384'
        !cp ../input/swin-large-models/swin_large_patch4_window12_384_22kto1k.pth /root/.cache/torch/hub/checkpoints/
    batch_size = 16
    
data = Data(batch_size=batch_size, img_size=img_size, n_split=n_split)
trainer = Trainer(num_epochs, lr, wd)

In [ ]:
for fold in range(n_split):
    print("---------------------------fold-%d-------------------------------"%fold)
    train_loss_list = trainer.train_fold(data, fold, model_name=model_name)
    plt.plot(train_loss_list)